Predicting TOALW, large errors from rought calculation

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
DS_a = xr.open_dataset('af') #dataset of auxhist
DS_w = xr.open_dataset('wf') #dataset of wrfout

In [3]:
#Temperature:
T = (DS_w.T.values + DS_w.T0) * ((DS_w.P.values + DS_w.PB.values)/DS_w.P0)**(DS_w.R_D/DS_w.CP)
T.shape

(71, 52, 36, 72)

In [4]:
st = DS_w.TSK.values #surface temp
st.shape

(71, 36, 72)

In [5]:
d_c = DS_w.TAU_OD.values #dust optical depth
d_c.shape 

(71, 52, 36, 72)

In [6]:
d_c2 = DS_w.TAU_OD2D.values
d_c2.shape

(71, 36, 72)

In [7]:
em = DS_w.EMISS.values #emissivity
em.shape 

(71, 36, 72)

In [8]:
lwr = DS_a.TOALW.values #top outer long wave radiation
lwr.shape

(71, 36, 72)

In [9]:
s_integral = np.zeros([71,36,72]) #special integral
for i in range(71):
    for j in range(36):
        for k in range(72):
            s_integral[i,j,k] = np.sum(np.exp(-d_c[i,:,j,k])*T[i,:,j,k]**4)

In [10]:
calc_out = em*st**4*np.exp(-d_c2) + s_integral #calcuating lwr according to equation
rcalc_out = calc_out.reshape(-1,1)
rlwr = lwr.reshape(-1,1)

In [11]:
from sklearn.linear_model import LinearRegression
lr1 = LinearRegression()
lr1.fit(rcalc_out,rlwr)
c_lwr_out = lr1.predict(rcalc_out)

In [12]:
from sklearn.metrics import mean_squared_error

lin_mse = mean_squared_error(rlwr, c_lwr_out)
lin_rmse = np.sqrt(lin_mse)
lin_rmse #wow, really bad error...

55.78044911324185

In [13]:
np.average(rlwr)

98.76447